## The Stability Experiment (Temperature Stress Test)

In [5]:
import sys
sys.path.append('..')

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model
from IPython.display import Markdown, display

In [6]:
# Properly load multi-line scenarios
file_path = "../data/scenarios.txt"

scenarios = []
current_scenario = ""

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        # New scenario starts
        if line.startswith("SCENARIO"):
            if current_scenario:
                scenarios.append(current_scenario.strip())
            current_scenario = line
        else:
            current_scenario += " " + line

    # Append last scenario
    if current_scenario:
        scenarios.append(current_scenario.strip())

print(f"Loaded {len(scenarios)} scenarios")
for s in scenarios:
    print("\n---\n", s)



Loaded 2 scenarios

---
 SCENARIO A: THE KANDY LANDSLIDE Location: Hanthana Tea Factory Details: "We are trapped. The access road is blocked. My uncle is stuck in the line rooms below, climbing a tree as water rises (Immediate Life Threat). However, inside the factory, we have a diabetic patient who has collapsed and needs insulin (Immediate Health Threat). We also have 40 people hungry."

---
 SCENARIO B: THE GAMPAHA HOSPITAL Location: Gampaha District General Hospital Details: "Power has failed in the ICU. We have 3 patients on ventilators with battery backup for 2 hours (Critical). Meanwhile, flood water is entering the ground floor ward where 50 elderly patients are bedridden (High Risk). We have one generator truck arriving in 30 mins, but it can only power one section."


In [14]:
# CoT auto-routes to reasoning model
reasoning_model = pick_model('groq', 'cot')
print(f'Using reasoning model: {reasoning_model}')
client_reasoning = LLMClient('groq', reasoning_model)


# Additional guidance
instruction = """You are a disaster response analyst working for the
Sri Lanka Disaster Management Center (DMC).

Your responsibility is to accurately classify incoming
messages during a national emergency.

You must:
- Carefully analyze the message
- Reason step-by-step internally before deciding
- Follow all classification rules strictly
- Avoid assumptions or hallucinations
- Produce only the required structured output

You MUST NOT explain your reasoning.
Only return the final classification result.
Analyze the message below and classify it according to
the disaster response rules.

CLASSIFICATION RULES

1. Intent:
- Rescue → Immediate danger to life
- Supply → Request for food, water, medicine, essentials
- Info → News reports or informational updates
- Other → Status updates or non-actionable messages

2. District:
- Only output a district if it is explicitly mentioned
- If no district is mentioned, output "None"

3. Priority:
- High → Rescue or urgent Supply
- Low → Info or Other

OUTPUT FORMAT (STRICT - NO EXTRA TEXT):
District: [Name or None] | Intent: [Rescue/Supply/Info/Other] | Priority: [High/Low]

"""


Using reasoning model: openai/gpt-oss-120b


In [23]:
print('Chaos Mode (Temprature = 1.0)')

for scenario in scenarios:
    print(f'\n{scenario}\n')

    prompt_text, spec = render(
        'cot_reasoning.v1',
        role='disaster response analyst',
        problem=scenario
    )

    full_prompt = f"""text: {prompt_text}
    instruction: {instruction}"""

    messages = [{'role': 'user', 'content': full_prompt}]
    for i in range(3):
        response = client_reasoning.chat(messages, temperature=1.0, max_tokens=spec.max_tokens)

        print(f'CoT Response : {i + 1}')
        print(response['text'])
        log_llm_call('openai', reasoning_model, 'cot', response['latency_ms'], response['usage'])

Chaos Mode (Temprature = 1.0)

SCENARIO A: THE KANDY LANDSLIDE Location: Hanthana Tea Factory Details: "We are trapped. The access road is blocked. My uncle is stuck in the line rooms below, climbing a tree as water rises (Immediate Life Threat). However, inside the factory, we have a diabetic patient who has collapsed and needs insulin (Immediate Health Threat). We also have 40 people hungry."

CoT Response : 1
District: Kandy | Intent: Rescue | Priority: High
CoT Response : 2
District: Kandy | Intent: Rescue | Priority: High
CoT Response : 3
District: Kandy | Intent: Rescue | Priority: High

SCENARIO B: THE GAMPAHA HOSPITAL Location: Gampaha District General Hospital Details: "Power has failed in the ICU. We have 3 patients on ventilators with battery backup for 2 hours (Critical). Meanwhile, flood water is entering the ground floor ward where 50 elderly patients are bedridden (High Risk). We have one generator truck arriving in 30 mins, but it can only power one section."

CoT Respo